In [1]:
import numpy as np
import matplotlib.pyplot as plt
from utils import *
X_train,Y_train = read_csv('train_emoji.csv')
X_test, Y_test = read_csv('tesss.csv')

In [2]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

In [3]:
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

In [4]:
def sentences_to_indices(X, word_to_index, max_len):
    m = X.shape[0]                                  
    X_indices = np.zeros((m,max_len))
    for i in range(m):               
        sentence_words = [w.lower() for w in X[i].split()]
        j = 0
        for w in sentence_words:
            X_indices[i, j] = word_to_index[w]
            j = j+1    
    return X_indices

In [5]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 1                  
    emb_dim = word_to_vec_map["cucumber"].shape[0]      
    emb_matrix = np.zeros((vocab_len, emb_dim))
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    embedding_layer.build((None,)) # Do not modify the "None".  This line of code is complete as-is.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [6]:
def predict_your_feel(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(input_shape, dtype='int32')
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices)    
    X = LSTM(128, return_sequences=True)(embeddings)
    X = Dropout(0.5)(X)
    X = LSTM(128, return_sequences=False)(X)
    X = Dropout(0.5)(X)
    X = Dense(5)(X)
    X = Activation('softmax')(X)
    model = Model(inputs=sentence_indices, outputs=X)
    return model

In [7]:
maxLen = len(max(X_train, key=len).split())
model = predict_your_feel((maxLen,), word_to_vec_map, word_to_index)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)

In [8]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
5/5 [==============================] - 5s 40ms/step - loss: 1.5939 - accuracy: 0.2176
Epoch 2/50
5/5 [==============================] - 0s 61ms/step - loss: 1.5389 - accuracy: 0.3117
Epoch 3/50
5/5 [==============================] - 0s 62ms/step - loss: 1.4843 - accuracy: 0.4076
Epoch 4/50
5/5 [==============================] - 0s 52ms/step - loss: 1.4314 - accuracy: 0.3813
Epoch 5/50
5/5 [==============================] - 0s 62ms/step - loss: 1.2786 - accuracy: 0.5700
Epoch 6/50
5/5 [==============================] - 0s 53ms/step - loss: 1.1896 - accuracy: 0.5626
Epoch 7/50
5/5 [==============================] - 0s 61ms/step - loss: 0.9980 - accuracy: 0.6441
Epoch 8/50
5/5 [==============================] - 0s 58ms/step - loss: 0.8074 - accuracy: 0.7009
Epoch 9/50
5/5 [==============================] - 0s 56ms/step - loss: 0.6898 - accuracy: 0.7251
Epoch 10/50
5/5 [==============================] - 0s 57ms/step - loss: 0.6200 - accuracy: 0.8006
Epoch 11/50
5/5 [============

In [9]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

2/2 [==============================] - 1s 27ms/step - loss: 0.6148 - accuracy: 0.8214

Test accuracy =  0.8214285969734192


In [10]:
# enter your sentences here
x_test = np.array(['please love me :)'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

please love me :) 😞


In [11]:
x_test = np.array(['life is wonderful'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

life is wonderful 😄
